# A1: Logic and lambda calculus

The lab is an exploration and learning exercise to be done in a group and also in discussion with the teachers and other students.

Before starting, please read the following instructions on [how to work on group assignments](https://github.com/sdobnik/computational-semantics/blob/master/README.md).

Write all your answers and the code in the appropriate boxes below.

## Translating English to logic and evaluating logic in a model

In [2]:
# This task needs NLTK and Jupyter Notebook (IPython package).
import nltk
from utils import display_latex, display_translation, display_tree, display, Markdown
read_expr = nltk.sem.Expression.fromstring

### 1. Propositional logic
Translate the following sentences into **propositional logic** and verify that they parse with Expression.fromstring(). Provide a key which shows how the propositional variables in your translation correspond to expressions of English. Briefly discuss any difficulties you encounter. (By difficulties we mean cases where the semantics of English expressions cannot be expressed to the same degree by the semantics of your logic representations, i.e. they do not mean the same). **[5 + 1 marks]**

In [3]:
propositions = {
    "If Alex plays the piano, she is smart.":
    read_expr('P -> Q'),
    
    "Alex is both smart and musical.":
    read_expr('R & Q'),
    
    "If Alex is not smart, Lydia is not happy.":
    read_expr('(-Q) -> S'),
    
    "If Alex or George plays the piano, they are musical.":
    read_expr('(P|T) -> R & U'),
    
    "George plays the piano.":
    read_expr('T'),
}

for text, semrep in propositions.items():
    display_translation(text, semrep)

"If Alex plays the piano, she is smart.": $(P\ \rightarrow\ Q)$

"Alex is both smart and musical.": $(R\ \land\ Q)$

"If Alex is not smart, Lydia is not happy.": $(-Q\ \rightarrow\ S)$

"If Alex or George plays the piano, they are musical.": $((P\ \lor\ T)\ \rightarrow\ (R\ \land\ U))$

"George plays the piano.": $T$

*Difficulties encountered:*

"Alex plays the piano" = P 
"Alex is smart" = Q
"Alex is musical" = R
"Lydia is not happy." = S
"George plays the piano." = T
"George is musical" = U

There is no such difficulty that hinders the transmition of the actual meaning. It is only a few words that we cannot easily integrate into the logic expressions, such as "both" or "she" (coreferences in general). Also, the sentence "George is musical" needs to be assigned to a total new variable due to the existence of "they" in the sentence "they are musical."

### 2. Valuation of Propositional logic

Imagine that we observe a world where 
- (i) Alex does not play the piano,
- (ii) Alex and Lydia are smart and musical,
- (iii) George is not musical,
- (iv) Lydia is happy,
- (v) George plays the piano. 

Translate this informal description of the world into a model by appropriately defining an evaluation function and evaluate the formulae from Question 1 in this model. Briefly comment the answers you get. **[5 + 1 marks]**.

In [4]:
#Mapping basic logic expressions (propositional symbols) to values (semantic symbols with semantic values)
val = nltk.Valuation([("P", False), ("Q", True), ("R", True), ("S", False), ("T", True), ("U", False), ("V", True), ("W", True)])

#TEST val['P'] Works

dom = set()
g = nltk.Assignment(dom)

#Instantiate model
m = nltk.Model(dom,val)

#model evaluation
print (m.evaluate ('P -> Q', g))
print (m.evaluate ('R & Q', g))
print (m.evaluate ('(-Q) -> S', g))
print (m.evaluate ('(P|T) -> R & U', g))
print (m.evaluate ('T', g))



#Lydia is smart = V, Lydia is musical = W

True
True
True
False
True


*Comments:*

'P -> Q' evaluation = True
Since we have Implication, P must be False or Q must be True. In this example, P is False and Q is True, which justifies the result.


'R & Q' evaluation = True
Since we have Conjuction, both R and Q must be True, which is the case.


'(-Q) -> S' evaluation = True
Similarly to the first case. "Q" is True in our mapping. Thus, -Q and S are False, but since at least -Q is False, the result is justified.


'(P|T) -> R & U' evaluation = False
Here the main operation is an Implication, with a Disjunction in the first part and a Conjuction in the second. The Conjuction is False because "R"(True) and "U"(False) should both be True or False, which is not the case. Thus, in order the evaluation to be True, the first part of the Implication must be False. However, it is not certain that the first part will always be False, due to the Disjunction.


'T' evaluation = True
Simply a True value.


### 3. Predicate logic *without quantifiers*

Translate the following sentences into predicate-argument formulae of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [14]:
sentences1 = {
    "Lydia likes George but Lydia doesn't like Alex": 
    read_expr(r'like(lydia,george) & -like(lydia,alex)'),
    
    "Lydia likes herself and so does George":
    read_expr(r'like(lydia,lydia) & like(george,george)'),
    
    "Charlie is an English pianist who plays a sonata":
    read_expr(r'english(charlie) & pianist(charlie) & play(charlie,sonata)'),
    
    "Lydia and George admire each other":
    read_expr(r'admire(lydia,george) & admire(george,lydia)'),
}

for text, semrep in sentences1.items():
    display_translation(text, semrep)


"Lydia likes George but Lydia doesn't like Alex": $(like(lydia,george)\ \land\ -like(lydia,alex))$

"Lydia likes herself and so does George": $(like(lydia,lydia)\ \land\ like(george,george))$

"Charlie is an English pianist who plays a sonata": $(english(charlie)\ \land\ pianist(charlie)\ \land\ play(charlie,sonata))$

"Lydia and George admire each other": $(admire(lydia,george)\ \land\ admire(george,lydia))$

*Difficulties encountered:*

The sentence "Lydia likes herself and so does George" is rather ambiguous to translate. Does the English sentence mean that "George likes Lydia" or "George likes himself" (in the same way Lydia likes herself) ?

Also, can we add spaces? (a_sonata)

In the third sentece, when used "be(charlie,english) & be(charlie,pianist) & play(charlie,sonata)", I had a problem in task 5, where the evaluation was "undefined".

### 4. First order logic with quantifiers

Translate the following sentences into quantified formulas of First Order Logic and verify that they parse with `Expression.fromstring()`. Briefly discuss any difficulties you encounter. **[4 + 1 marks]**

In [25]:
sentences2 = {
    "Charlie knows a woman who likes George":
    read_expr('(exists x.(know(charlie, x) & like(x, george)))'),
    #'exists x.(charlie(x) & know(x,woman) & like(woman,george))' -> first attempt 
    "George admires everybody and Lydia admires nobody":
    read_expr('all x.(admire(george,x) & -admire(lydia,x))'),
    #'all x.(person(x) -> admire(george,x) & -admire(lydia,x))' -> first attempt, undefined in task 5
    "Nobody admires everybody":
    read_expr('all x. exists y. -admire(x,y)'),

    "Exactly one musician plays everything Alex wrote":
    read_expr('exists x. all y. (musician(x) & play(x,y) & write(Alex,y))'),
}

for text, semrep in sentences2.items():
    display_translation(text, semrep)

"Charlie knows a woman who likes George": $\exists\ x.(know(charlie,x)\ \land\ like(x,george))$

"George admires everybody and Lydia admires nobody": $\forall\ x.(admire(george,x)\ \land\ -admire(lydia,x))$

"Nobody admires everybody": $\forall\ x.\exists\ y.-admire(x,y)$

"Exactly one musician plays everything Alex wrote": $\exists\ x.\forall\ y.(musician(x)\ \land\ play(x,y)\ \land\ write(Alex,y))$

*Difficulties encountered:*

I used 'exists x.(charlie(x) & know(x,woman) & like(woman,george))' for the first sentence, but then in task 5 it yeld an error. ???

The second translation can be ambiguous because "George admires everybody and Lydia admires nobody" is a bit different than "George admires everybody and Lydia does not admire everybody", which means that Lydia could admire some people.

For the third translation, the interpretation of the First-Order Logic could be "All do not admire someone" (so some admire someone?), which maybe has a slightly different meaning than "Nobody admires everybody" (so some people admire some people?).

Finally, the only thing that is quite ambiguous in the last translation is the word "Exactly". The interpretation that derives from the formula does not really affect the final meaning, however it would probably be problematic in case we need to be explicit with the meaning we want to transmit.

### 5. Valuation of first order logic

We observe a world with entities Lydia, George, Alex, Charlie and Bertie, sonata, etude, prelude, waltz, scherzo.

1. Lydia likes Lydia, George, Alex and Charlie. George likes Lydia, Bertie and George. Alex likes Alex. Charlie likes Lydia, George, Alex, Charlie and Bertie. Bertie likes Alex.
2. Lydia, George, Alex, Charlie and Bertie are English.
3. Charlie and Bertie are pianists.
4. Charlie plays a sonata, an etude and a waltz. Bertie plays a waltz and a scherzo. Lydia plays an etude, a prelude and a waltz.
5. Lydia admires Lydia, Charlie and Bertie. George admires Lydia, George, Alex, Charlie and Bertie. Alex admires Lydia, Alex and Bertie. Charlie admires George and Bertie. Bertie admires Lydia, George, Alex, Charlie and Bertie.
6. Lydia knows Lydia, George, Alex, Charlie and Bertie. George knows Lydia, George and Bertie. Alex knows Lydia, Alex and Bertie. Charlie knows George, Charlie and Bertie. Bertie knows Lydia, George, Alex, Charlie and Bertie.
7. Lydia, Alex and Charlie are women.
8. George and Bertie are men.
9. Alex wrote a sonata, an etude an a waltz.
10. Lydia, Alex, Charlie and Bertie are musicians.

Translate this informal description of the world into a model and evaluate the formulae from Questions 3 and 4 in this model. Briefly comment on the answers you get **[3 + 2 marks]**.

In [26]:
entities = set(['p','t','e','h','r','s','u','l','w','c'])

assign = """
lydia => p
george => t
alex => e
charlie => h
bertie => r
sonata => s
etude => u
prelude => l
waltz => w
scherzo => c
man => {t,r}
woman => {p,e,h}
pianist => {h,r}
musician => {p,e,h,r}
english => {p,t,e,h,r}
write => {(e,s), (e,u), (e,w)}
like => {(p,p), (p,t), (p,e), (p,h), (t,p), (t,r), (t,t), (e,e), (h,p), (h,t), (h,e), (h,h), (h,r), (r,e)}
play => {(h,s), (h,u), (h,w), (r,w), (r,c), (p,u), (p,l), (p,w)}
admire => {(p,p), (p,h), (p,r), (t,p), (t,t), (t,e), (t,h), (t,r), (h,t), (h,r), (r,p), (r,t), (r,e), (r,h), (r,r)}
know => {(p,p), (p,t), (p,e), (p,h), (p,r), (t,p), (t,t), (t,r), (e,p), (e,e), (e,r), (h,t), (h,h), (h,r), (r,p), (r,t), (r,e), (r,h), (r,r)}
"""

val2 = nltk.Valuation.fromstring(assign)

g2 = nltk.Assignment(entities)
m2 = nltk.Model(entities, val2)

# sentences from question 3
for text, semrep in sentences1.items():
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))

# sentences from question 4
for text, semrep in sentences2.items():
    print(m2.evaluate(str(semrep), g2))
    display_latex(semrep)
    display(Markdown('----'))


False


$(like(lydia,george)\ \land\ -like(lydia,alex))$

----

True


$(like(lydia,lydia)\ \land\ like(george,george))$

...

$\forall\ x.\exists\ y.-admire(x,y)$

----

False


$\exists\ x.\forall\ y.(musician(x)\ \land\ play(x,y)\ \land\ write(Alex,y))$

----

*Comments on the answers:*

`Answers here`

## Lambda calculus

In [ ]:
from nltk.grammar import FeatureGrammar

### 6. Function application and $\beta$-reduction
In the following examples some code has been deleted and replaced with `<????>`. What has been deleted? Verify that your answer is correct. **[4 marks]**

In [90]:
e1 = read_expr(r'\x.(like(x,rob))')
e2 = read_expr(r'pip')
e3 = nltk.sem.ApplicationExpression(e1,e2) 
display_latex(e3.simplify())
# with result like(pip,rob).
display_latex(read_expr(r"like(pip,rob)"))

e1 = read_expr(r'\P.(P(pip))') #nested
e2 = read_expr(r'\x.play(x,scherzo)') 
e3 = nltk.sem.ApplicationExpression(e1,e2)
display_latex(e3.simplify())
# with result play(pip,scherzo).
display_latex(read_expr(r"play(pip,scherzo)"))

e1 = read_expr(r'\P.exists x.(woman(x) & P(x))')
e2 = read_expr(r'\x.play(x,etude)') 
e3 = nltk.sem.ApplicationExpression(e1,e2) 
display_latex(e3.simplify())
# with result exists x.(woman(x) & play(x,etude)).
display_latex(read_expr(r"exists x.(woman(x) & play(x,etude))"))


e1 = read_expr(r'\X x.X(\z2.like(x,z2))')
e2 = read_expr(r'\P.all x. (musician(x) -> P(x))')
e3 = nltk.sem.ApplicationExpression(e1,e2) 
display_latex(e3.simplify())
# with result \x.all z2.(musician(z2) -> like(x,z2)).
display_latex(read_expr(r"\x.all z2.(musician(z2) -> like(x,z2))")) 

#Why is my result z16?

$like(pip,rob)$

$like(pip,rob)$

$play(pip,scherzo)$

$play(pip,scherzo)$

$\exists\ x.(woman(x)\ \land\ play(x,etude))$

$\exists\ x.(woman(x)\ \land\ play(x,etude))$

$\lambda\ x.\forall\ z_{17}.(musician(z_{17})\ \rightarrow\ like(x,z_{17}))$

$\lambda\ x.\forall\ z_{2}.(musician(z_{2})\ \rightarrow\ like(x,z_{2}))$

### 7. Extending the grammar

Extend the grammar simple_sem.fcfg that comes with NLTK `(~/nltk_data/grammars/book_grammars/)` so that it will cover the following sentences:

- no man gives a bone to a dog **[4 marks]**
- no man gives a bone to the dog **[4 marks]**
- a boy and a girl chased every dog **[2 marks]**
- every dog chased a boy and a girl **[2 marks]**
- a brown cat chases a white dog **[4 marks]**

The last example includes adjectives. Several different kinds of adjectives are discussed in the literature [(cf. Kennedy, 2012)](http://semantics.uchicago.edu/kennedy/docs/routledge.pdf). In this example we have an intersective adjective. The denotiation we want for "brown cat" is a a set that we get by intersecting the set of individuals that are brown and the set of individuals that are cats.

C. Kennedy. Adjectives. In G. Russell, editor, The Routledge Companion to Philosophy of Language, chapter 3.3, pages 328–341. Routledge, 2012.

The original grammar is included in the code below as a string.

In [1]:
fcfg_string_orginal = r"""
% start S
############################
# Grammar Rules
#############################

S[SEM = <?subj(?vp)>] -> NP[NUM=?n,SEM=?subj] VP[NUM=?n,SEM=?vp]

NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det]  Nom[NUM=?n,SEM=?nom]
#NP[LOC=?l,NUM=?n,SEM=?np] -> PropN[LOC=?l,NUM=?n,SEM=?np]

Nom[NUM=?n,SEM=?nom] -> N[NUM=?n,SEM=?nom]

#VP[NUM=?n,SEM=?v] -> IV[NUM=?n,SEM=?v]
VP[NUM=?n,SEM=<?v(?obj)>] -> TV[NUM=?n,SEM=?v] NP[SEM=?obj]
VP[NUM=?n,SEM=<?v(?obj,?pp)>] -> DTV[NUM=?n,SEM=?v] NP[SEM=?obj] PP[+TO,SEM=?pp]

PP[+TO, SEM=?np] -> P[+TO] NP[SEM=?np]

#############################
# Lexical Rules
#############################

PropN[-LOC,NUM=sg,SEM=<\P.P(angus)>] -> 'Angus'
PropN[-LOC,NUM=sg,SEM=<\P.P(cyril)>] -> 'Cyril'
PropN[-LOC,NUM=sg,SEM=<\P.P(irene)>] -> 'Irene'
 
Det[NUM=sg,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'every'
Det[NUM=pl,SEM=<\P Q.all x.(P(x) -> Q(x))>] -> 'all'
Det[SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'some'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'a'
Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'an'

N[NUM=sg,SEM=<\x.man(x)>] -> 'man'
N[NUM=sg,SEM=<\x.girl(x)>] -> 'girl'
N[NUM=sg,SEM=<\x.boy(x)>] -> 'boy'
N[NUM=sg,SEM=<\x.bone(x)>] -> 'bone'
N[NUM=sg,SEM=<\x.ankle(x)>] -> 'ankle'
N[NUM=sg,SEM=<\x.dog(x)>] -> 'dog'
N[NUM=pl,SEM=<\x.dog(x)>] -> 'dogs'

IV[NUM=sg,SEM=<\x.bark(x)>,TNS=pres] -> 'barks'
IV[NUM=pl,SEM=<\x.bark(x)>,TNS=pres] -> 'bark'
IV[NUM=sg,SEM=<\x.walk(x)>,TNS=pres] -> 'walks'
IV[NUM=pl,SEM=<\x.walk(x)>,TNS=pres] -> 'walk'
TV[NUM=sg,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chases'
TV[NUM=pl,SEM=<\X x.X(\ y.chase(x,y))>,TNS=pres] -> 'chase'
TV[NUM=sg,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'sees'
TV[NUM=pl,SEM=<\X x.X(\ y.see(x,y))>,TNS=pres] -> 'see'
TV[NUM=sg,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bites'
TV[NUM=pl,SEM=<\X x.X(\ y.bite(x,y))>,TNS=pres] -> 'bite'
DTV[NUM=sg,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'gives'
DTV[NUM=pl,SEM=<\Y X x.X(\z.Y(\y.give(x,y,z)))>,TNS=pres] -> 'give'

P[+to] -> 'to'
"""

Write your extension of this grammar here:

In [3]:
import sys
from nltk.grammar import FeatureGrammar
fcfg_string = fcfg_string_orginal + r"""
## Your answers here
TV[SEM=<\X x.X(\ y.chase(x,y))>,TNS=past] -> 'chased'
N[NUM=sg,SEM=<\x.cat(x)>] -> 'cat'
CONJ -> "and"
ADJ[SEM=<\x.brown(x)>] -> "brown"
ADJ[SEM=<\x.white(x)>] -> "white"

Det[NUM=sg,SEM=<\P Q.-all x.(P(x) -> Q(x))>] -> 'no' 
# oposite to "all" and singular

Det[NUM=sg,SEM=<\P Q.exists x.(P(x) & Q(x))>] -> 'the' 
#similarly to "a" for these sentences, otherwice "the" could be both singular and plural

NP[NUM=?n,SEM=<?det(?nom)> ] -> Det[NUM=?n,SEM=?det] Nom[NUM=?n,SEM=?nom] 
# -> a boy and a girl


# NP[???] -> NP[???] CONJ NP[???]
# TV[???] -> ???
# NP[???] -> Det[???] ADJ[???] Nom[???]
"""

# Load `fcfg_string` as a feature grammar:
syntax = FeatureGrammar.fromstring(fcfg_string)

Run the code below without errors:

In [5]:
# remove sentences if you couldn't find answer for them
sentences = [
    'no man gives a bone to a dog'
    'no man gives a bone to the dog',
    'a boy and a girl chased every dog',
    'every dog chased a boy and a girl',
    'a brown cat chases a white dog',
]
for results in nltk.interpret_sents(sentences, syntax):
    for (synrep, semrep) in results:
        display(Markdown('----'))
        display_latex(semrep) # prints the SEM feature of a tree
        display_tree(synrep) # show the parse tree

----

$-\forall\ x.(man(x)\ \rightarrow\ \exists\ z_{2}.(dog(z_{2})\ \land\ \exists\ z_{1}.(bone(z_{1})\ \land\ give(x,z_{1},z_{2}))))$

OSError: [Errno 86] Bad CPU type in executable: '/usr/local/bin/gs'

If you are working with iPython which is also running behind Jupyter notebooks and you are changing grammars and want to rerun a new version without restarting you may find `nltk.data.clear_cache()` useful.

## Marks

This part of the assignment has a total of 47 marks.